In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import pandas as pd
import numpy as np

import time

import torch

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
BATCH_SIZE = 32
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
def fit(model, save_dir, train_set, test_set, forget_set=None):
    
    os.makedirs(save_dir, exist_ok=True)
    
    optimizer = torch.optim.Adam(model.parameters())
    error = torch.nn.CrossEntropyLoss()
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):
        
        # train
        
        accs = list()
        
        train_time = 0
        
        model.train()
        
        start_time = time.time()
        
        for x, y in train_loader:
            optimizer.zero_grad()
            output = model(x.cuda())
            y = y.cuda()
            loss = error(output, y)
            loss.backward()
            optimizer.step()
            
            train_time += time.time() - start_time
            
            predicted = torch.argmax(output.data, dim=-1)
            accs.append((predicted == y).float().mean().detach().cpu().numpy())
            
            start_time = time.time()
            
        train_times.append(train_time)
        train_accs.append(np.mean(accs))
        
        # eval
            
        model.eval()
        with torch.no_grad():
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            if forget_set is not None:

                x, y = forget_set.tensors

                for i in range(0, x.shape[0], BATCH_SIZE):

                    output = model(x[i:i+BATCH_SIZE].cuda())

                    predicted = torch.argmax(output.data, dim=-1)
                    accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

                forget_accs.append(np.mean(accs))
        
        # save
        torch.save(model.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))
        

    return train_times, train_accs, test_accs, forget_accs

# origin train

In [5]:
train_set, test_set = load_original_dataset(DATA_DIR)

In [6]:
model = CNN().cuda()

In [7]:
# save
torch.save(model.state_dict(), 'weights/init.pt')

In [8]:
train_times, train_accs, test_accs, _ = fit(model, 'weights/original', train_set, test_set)

In [9]:
df = pd.DataFrame(zip(train_times, train_accs, test_accs), columns=['train_time', 'train_acc', 'test_acc'])
df['epoch'] = range(1, EPOCHS+1)
df.set_index('epoch', inplace=True)

df.to_csv('results/original.csv')

df

,train_time,train_acc,test_acc
epoch,,,
1,13.418311,0.853767,0.971845
2,11.025154,0.943200,0.980032
3,10.974817,0.952717,0.984924
4,10.965304,0.955550,0.983526
5,11.331731,0.958683,0.987220


# naive retrain

In [10]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
        
    model.load_state_dict(torch.load('./weights/init.pt'))
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/naive_retrain/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [11]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/naive_retrain.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1       10.982173   0.849441  0.975240    0.975245
           2       11.042801   0.940968  0.984325    0.984281
           3       11.298384   0.950498  0.986422    0.986258
           4       11.589271   0.953731  0.985323    0.984657
           5       11.134772   0.957065  0.987819    0.987858
10         1       10.033020   0.844658  0.976138    0.975299
           2        9.801108   0.938648  0.982129    0.980539
           3        9.861707   0.947040  0.982528    0.981911
           4       10.597235   0.954320  0.988319    0.987587
           5       10.060778   0.957210  0.987320    0.986153
20         1        8.655482   0.841062  0.971845    0.970203
           2        9.253140   0.938833  0.983427    0.979424
           3        8.767224   0.950083  0.985523    0.982876
           4        8.704133   0.955500  0.985723    0.982286
           5        8.727246   0.956688  0.987420    0.986056
30         1        7.578672   0.833651  0.975339    0.971961
           2        7.565557   0.938858  0.980331    0.978346
           3        7.587089   0.946051  0.983327    0.980879
           4        7.550887   0.953720  0.985823    0.984197
           5        7.447364   0.955650  0.983526    0.980094
40         1        6.533855   0.810056  0.965954    0.964428
           2        6.295882   0.931528  0.980631    0.979451
           3        6.413925   0.946694  0.979433    0.978804
           4        6.735131   0.951639  0.980931    0.981215
           5        6.998962   0.956056  0.984026    0.983243
50         1        5.508229   0.777781  0.969349    0.966702
           2        5.669635   0.925694  0.976438    0.975345
           3        5.578954   0.940835  0.979633    0.977643
           4        5.495699   0.947038  0.984824    0.981889
           5        5.561973   0.951407  0.981829    0.979192
60         1        4.244047   0.753167  0.956370    0.952191
           2        4.413632   0.921083  0.969649    0.965621
           3        4.406580   0.938042  0.977336    0.972575
           4        4.363652   0.945417  0.983726    0.979703
           5        4.371818   0.949458  0.984724    0.981354
70         1        3.093799   0.695785  0.942192    0.937308
           2        3.145003   0.891681  0.953474    0.950992
           3        3.207841   0.918427  0.968650    0.966079
           4        3.213466   0.932162  0.975240    0.971133
           5        3.236058   0.938445  0.978235    0.975669
80         1        1.950946   0.695417  0.939297    0.934432
           2        2.131942   0.883000  0.956669    0.954926
           3        2.276719   0.914250  0.961362    0.959029
           4        1.976451   0.930083  0.969449    0.965889
           5        2.116694   0.934500  0.970847    0.968078
90         1        0.863657   0.458389  0.884285    0.873657
           2        1.140768   0.840742  0.942292    0.937812
           3        0.983114   0.890709  0.954373    0.948573
           4        0.955004   0.910595  0.963359    0.958630
           5        0.995988   0.923797  0.967552    0.963518
99         1        0.061361   0.114583  0.203574    0.204176
           2        0.073275   0.196181  0.410942    0.401368
           3        0.072175   0.350694  0.542232    0.537313
           4        0.068925   0.522569  0.697484    0.682172
           5        0.067896   0.638889  0.748003    0.736146